## Import Libraries

In [1]:
from pymongo import MongoClient
import json
from bson import ObjectId

In [2]:
import pprint

In [3]:
MONGODB_HOST = 'localhost'
MONGODB_PORT = 27017
DBS_NAME = 'mydb'
COLLECTION_NAME = 'projects'
FIELDS = "FeatureCollection"

In [936]:
def dashboard_projects():
    connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
    collection = connection[DBS_NAME][COLLECTION_NAME]
    projects = collection.find_one({"type": "FeatureCollection"}, {"_id":0})

    return projects

In [937]:
dashboard_projects()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.965463853978, -37.8077644149636]},
   'properties': {'FIELD1': 0,
    'Theme': 'Place Of Assembly',
    'Sub Theme': 'Art Gallery/Museum',
    'Feature Name': 'Old Melbourne Gaol Crime & Justice Experience'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.953478052064, -37.8242574266145]},
   'properties': {'FIELD1': 1,
    'Theme': 'Place Of Assembly',
    'Sub Theme': 'Art Gallery/Museum',
    'Feature Name': 'Polly Woodside'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.967252924593, -37.8266045432686]},
   'properties': {'FIELD1': 2,
    'Theme': 'Place Of Assembly',
    'Sub Theme': 'Art Gallery/Museum',
    'Feature Name': 'Australian Centre for Contemporary Art'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.968837288153, -37.8219947335768]},
   'properties'

In [295]:
# Call all documents
connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
collection = connection[DBS_NAME][COLLECTION_NAME]
projects2 = collection.find_one({"type": "FeatureCollection"}, {"_id":0})

In [321]:
client = MongoClient()
db = client.mydb
collection = db.projects

In [319]:
from bson.son import SON
from pymongo import MongoClient

db = MongoClient()
latitude = -37.44609999
longitude = 144.96923385414
max_distance = 1000 #meters    

In [903]:
location = [144.965234, -37.8076]

In [755]:
db.collection.drop_indexes()

In [756]:
for idx in db.collection.list_indexes():
    print (idx)

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'mydb.collection')])


In [757]:
db.collection.create_index([( "features.geometry.coordinates", "2dsphere" )])

'features.geometry.coordinates_2dsphere'

In [785]:
## Unwind search

pipeline = [{"$unwind": "$features"},
            {"$match":{'features.geometry.coordinates':[144.965463853978, -37.8077644149636]}},
            {"$group": {"_id": "$features"}}]

pprint.pprint(list(collection.aggregate(pipeline)))

[]


In [787]:
pipeline2 = [{"$unwind": "$features"},
            {"$match":{'features.properties.Sub Theme' : 'Art Gallery/Museum'}},
            {"$group": {"_id": "$features"}}]

pprint.pprint(list(collection.aggregate(pipeline2)))

[{'_id': {'geometry': {'coordinates': [144.969070431323, -37.8176107327202],
                       'type': 'Point'},
          'properties': {'FIELD1': 17,
                         'Feature Name': 'Australian Centre For The Moving '
                                         'Image (ACMI)',
                         'Sub Theme': 'Art Gallery/Museum',
                         'Theme': 'Place Of Assembly'},
          'type': 'Feature'}},
 {'_id': {'geometry': {'coordinates': [144.971521914952, -37.80460262432],
                       'type': 'Point'},
          'properties': {'FIELD1': 16,
                         'Feature Name': 'Royal Exhibition Building',
                         'Sub Theme': 'Art Gallery/Museum',
                         'Theme': 'Place Of Assembly'},
          'type': 'Feature'}},
 {'_id': {'geometry': {'coordinates': [144.979471339826, -37.8144604899837],
                       'type': 'Point'},
          'properties': {'FIELD1': 15,
                         'Feature

## Working code

In [904]:
pipeline = [
        {'$geoNear': {
            'near' : { 'type' : "Point", 'coordinates' : location },
             'distanceField': "dist.calculated",
             'maxDistance': 100,
             'includeLocs': "dist.location",
              'key': 'features.geometry.coordinates',
             'uniqueDocs': False,
             'spherical': True}},
    { "$unwind" : "$features" },
    { "$redact" : {   
        "$cond" : {
            "if" : { "$eq" : [ { "$cmp" : [ "$features.geometry.coordinates", "$dist.location" ] }, 0 ] },
            "then" : "$$KEEP",
            "else" : "$$PRUNE"
        }
    }
}
]

pprint.pprint(list(collection.aggregate(pipeline)))

[{'_id': ObjectId('5ca63a65122b9a6192720138'),
  'dist': {'calculated': 27.27001406965783,
           'location': [144.965463853978, -37.8077644149636]},
  'features': {'geometry': {'coordinates': [144.965463853978,
                                            -37.8077644149636],
                            'type': 'Point'},
               'properties': {'FIELD1': 0,
                              'Feature Name': 'Old Melbourne Gaol Crime & '
                                              'Justice Experience',
                              'Sub Theme': 'Art Gallery/Museum',
                              'Theme': 'Place Of Assembly'},
               'type': 'Feature'},
  'type': 'FeatureCollection'}]


### Check

In [820]:
a1 = [144.9654, -37.8076]
a2 = [144.9656988, -37.80619915]

distance_finder(a1, a2)

157.69509130741474 m


## Implement Automation

In [6]:
click = [144.9607616045045, -37.811441609877804]

In [7]:
location = [144.965234, -37.8076]

In [8]:
pipeline = [
        {'$geoNear': {
            'near' : { 'type' : "Point", 'coordinates' : click },
             'distanceField': "dist.calculated",
             'maxDistance': 500,
             'includeLocs': "dist.location",
              'key': 'features.geometry.coordinates',
             'uniqueDocs': False,
             'spherical': True}},
    { "$unwind" : "$features" },
    { "$redact" : {   
        "$cond" : {
            "if" : { "$eq" : [ { "$cmp" : [ "$features.geometry.coordinates", "$dist.location" ] }, 0 ] },
            "then" : "$$KEEP",
            "else" : "$$PRUNE"
        }
    }
}
]

pprint.pprint(list(collection.aggregate(pipeline)))

[{'_id': ObjectId('5cb419fc9a2c9b0218cbbbd2'),
  'dist': {'calculated': 68.10091452076718,
           'location': [144.9602499, -37.81190077]},
  'features': {'geometry': {'coordinates': [144.9602499, -37.81190077],
                            'type': 'Point'},
               'properties': {'Base property ID': 106018,
                              'Block ID': 74,
                              'CLUE small area': 'Melbourne (CBD)',
                              'Census year': 2015,
                              'FIELD1': 67,
                              'Industry (ANZSIC4) code': 4520,
                              'Industry (ANZSIC4) description': 'Pubs, Taverns '
                                                                'and Bars',
                              'Number of seats': 50,
                              'Property ID': 106018,
                              'Seating type': 'Seats - Indoor',
                              'Street address': '395-397 Little Lonsdale '
      

In [9]:
connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
collection = connection[DBS_NAME][COLLECTION_NAME]
pprint.pprint(list(collection.aggregate(pipeline)))
connection.close()

[{'_id': ObjectId('5cb419fc9a2c9b0218cbbbd2'),
  'dist': {'calculated': 68.10091452076718,
           'location': [144.9602499, -37.81190077]},
  'features': {'geometry': {'coordinates': [144.9602499, -37.81190077],
                            'type': 'Point'},
               'properties': {'Base property ID': 106018,
                              'Block ID': 74,
                              'CLUE small area': 'Melbourne (CBD)',
                              'Census year': 2015,
                              'FIELD1': 67,
                              'Industry (ANZSIC4) code': 4520,
                              'Industry (ANZSIC4) description': 'Pubs, Taverns '
                                                                'and Bars',
                              'Number of seats': 50,
                              'Property ID': 106018,
                              'Seating type': 'Seats - Indoor',
                              'Street address': '395-397 Little Lonsdale '
      

In [4]:
connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
collection = connection[DBS_NAME][COLLECTION_NAME]
finding = collection.find_one( { "_id": ObjectId('5cc683b30e00cbd80dd912ad') }, {"_id":0})
connection.close()
finding

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9651532, -37.8134944]},
   'properties': {'sensor_id': 1,
    'sensor_description': 'Bourke Street Mall (North)',
    'sensor_name': 'Bou292_T',
    'installation_date': '24/03/2009',
    'status': 'A',
    'note': '',
    'direction_1': 'West',
    'direction_2': 'East'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9651672, -37.81380667]},
   'properties': {'sensor_id': 2,
    'sensor_description': 'Bourke Street Mall (South)',
    'sensor_name': 'Bou283_T',
    'installation_date': '30/03/2009',
    'status': 'A',
    'note': '',
    'direction_1': 'East',
    'direction_2': 'West'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [144.9642949, -37.81101523]},
   'properties': {'sensor_id': 3,
    'sensor_description': 'Melbourne Central',
    'sensor_name': 'Swa295_T',
    'installation_date': '25/03/2009',
    's

In [27]:
connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
collection = connection[DBS_NAME][COLLECTION_NAME]
finding = collection.find_one()
connection.close()
finding

{'_id': ObjectId('5ca5a19f9a2c9b0218cbb6e5'),
 'school state': 'NY',
 'resource type': 'Books',
 'poverty level': 'highest poverty',
 'highest poverty': 308,
 'total donations': 'competed',
 'date posted': '08/03/2007'}

In [25]:
for i in collection.find():
    pprint.pprint(i)

{'_id': ObjectId('5ca5a19f9a2c9b0218cbb6e5'),
 'date posted': '08/03/2007',
 'highest poverty': 308,
 'poverty level': 'highest poverty',
 'resource type': 'Books',
 'school state': 'NY',
 'total donations': 'competed'}
{'_id': ObjectId('5ca823f39a2c9b0218cbb753'),
 'features': [{'geometry': {'coordinates': [144.965463853978,
                                            -37.8077644149636],
                            'type': 'Point'},
               'properties': {'FIELD1': 0,
                              'Feature Name': 'Old Melbourne Gaol Crime & '
                                              'Justice Experience',
                              'Sub Theme': 'Art Gallery/Museum',
                              'Theme': 'Place Of Assembly'},
               'type': 'Feature'},
              {'geometry': {'coordinates': [144.953478052064,
                                            -37.8242574266145],
                            'type': 'Point'},
               'properties': {'FIELD1': 1,

                            'type': 'Point'},
               'properties': {'Base property ID': 101113,
                              'Block ID': 48,
                              'CLUE small area': 'Melbourne (CBD)',
                              'Census year': 2015,
                              'FIELD1': 52,
                              'Industry (ANZSIC4) code': 4520,
                              'Industry (ANZSIC4) description': 'Pubs, Taverns '
                                                                'and Bars',
                              'Number of seats': 80,
                              'Property ID': 101113,
                              'Seating type': 'Seats - Indoor',
                              'Street address': '59-63 Bourke Street',
                              'Trading name': 'Club Asuka'},
               'type': 'Feature'},
              {'geometry': {'coordinates': [144.9711999, -37.81285806],
                            'type': 'Point'},
            

                              'FIELD1': 120,
                              'Industry (ANZSIC4) code': 4520,
                              'Industry (ANZSIC4) description': 'Pubs, Taverns '
                                                                'and Bars',
                              'Number of seats': 100,
                              'Property ID': 105933,
                              'Seating type': 'Seats - Indoor',
                              'Street address': '425-427 Little Collins Street',
                              'Trading name': 'The Irish Times'},
               'type': 'Feature'},
              {'geometry': {'coordinates': [144.9564078, -37.81703156],
                            'type': 'Point'},
               'properties': {'Base property ID': 105392,
                              'Block ID': 42,
                              'CLUE small area': 'Melbourne (CBD)',
                              'Census year': 2015,
                              'FIELD1': 1

                            'type': 'Point'},
               'properties': {'Base property ID': 110733,
                              'Block ID': 752,
                              'CLUE small area': 'Southbank',
                              'Census year': 2015,
                              'FIELD1': 182,
                              'Industry (ANZSIC4) code': 4520,
                              'Industry (ANZSIC4) description': 'Pubs, Taverns '
                                                                'and Bars',
                              'Number of seats': 20,
                              'Property ID': 108653,
                              'Seating type': 'Seats - Indoor',
                              'Street address': '100 St Kilda Road',
                              'Trading name': 'Mezz Bar'},
               'type': 'Feature'},
              {'geometry': {'coordinates': [144.9814116, -37.84752524],
                            'type': 'Point'},
               'prop